In [ ]:
!nvidia-smi

Thu Jul  6 20:02:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -qU transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
train = pd.read_excel('/content/drive/MyDrive/abuse/data/train.xlsx')
test = pd.read_excel('/content/drive/MyDrive/abuse/data/test.xlsx')
test.head()

,tweet,label
0,باشه❤,0
1,از ۸۶ به اینور کلا تحریمه از طرف من,0
2,بیوش:),0
3,😒نخواستیم اصلن,0
4,روم به دیفال,0


In [ ]:
from sklearn.utils import shuffle
train = shuffle(train)
test = shuffle(test)

In [ ]:
test.head()

,tweet,label
3309,واقعا چرا؟!!!,0
7941,روو یکی کراش زدم ولی نمیگم کیه.خیلی هم کراش زدم,0
123,منفیه؟,0
8550,چون خدا غرور داده به ما,0
3580,کی تموم میشه این طلسمی که توش گرفتاریم این راه...,0


In [ ]:
!pip install simpletransformers -q

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 3,
    "evaluate_during_training_steps": 1000,
    "train_batch_size": 128,
    "eval_batch_size": 64,
}

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


train_df = train[1000:]

# Preparing eval data
eval_df = train[0:1000]

# Optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs=1
model_args.labels_list = [0, 1]

model_args.overwrite_output_dir =  True
# "xlmroberta", "pedramyazdipoor/persian_xlm_roberta_large", args=model_args

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "HooshvareLab/bert-base-parsbert-uncased", args=model_args
)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)




Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifica

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/22335 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/2792 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

In [ ]:
test_sents = test["tweet"].values.tolist()
test_labels = test["label"].values.tolist()

In [ ]:
predictions, raw_outputs = model.predict(test_sents)

  0%|          | 0/10003 [00:00<?, ?it/s]

  0%|          | 0/1251 [00:00<?, ?it/s]

In [ ]:
# calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# calculate score
print('accuracy_score: %.3f' % accuracy_score(predictions, test_labels))
print('precision_score: %.3f' % precision_score(predictions, test_labels))
print('recall_score: %.3f' % recall_score(predictions, test_labels))
print('F-Measure: %.3f' % f1_score(predictions, test_labels))


accuracy_score: 0.981
precision_score: 0.832
recall_score: 0.969
F-Measure: 0.896


In [ ]:
test_sents = train["tweet"].values.tolist()
test_labels = train["label"].values.tolist()
predictions, raw_outputs = model.predict(test_sents)

  0%|          | 0/23335 [00:00<?, ?it/s]

  0%|          | 0/2917 [00:00<?, ?it/s]

In [ ]:
sum(test_labels)

2305

In [ ]:
# calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# calculate score
print('accuracy_score: %.3f' % accuracy_score(predictions, test_labels))
print('precision_score: %.3f' % precision_score(predictions, test_labels))
print('recall_score: %.3f' % recall_score(predictions, test_labels))
print('F-Measure: %.3f' % f1_score(predictions, test_labels))

accuracy_score: 0.998
precision_score: 0.990
recall_score: 0.987
F-Measure: 0.989


In [ ]:
%cp -av "/content/outputs" "/content/drive/MyDrive/abuse/"

'/content/outputs' -> '/content/drive/MyDrive/abuse/outputs'
'/content/outputs/checkpoint-2000' -> '/content/drive/MyDrive/abuse/outputs/checkpoint-2000'
'/content/outputs/checkpoint-2000/config.json' -> '/content/drive/MyDrive/abuse/outputs/checkpoint-2000/config.json'
'/content/outputs/checkpoint-2000/pytorch_model.bin' -> '/content/drive/MyDrive/abuse/outputs/checkpoint-2000/pytorch_model.bin'
'/content/outputs/checkpoint-2000/tokenizer_config.json' -> '/content/drive/MyDrive/abuse/outputs/checkpoint-2000/tokenizer_config.json'
'/content/outputs/checkpoint-2000/special_tokens_map.json' -> '/content/drive/MyDrive/abuse/outputs/checkpoint-2000/special_tokens_map.json'
'/content/outputs/checkpoint-2000/vocab.json' -> '/content/drive/MyDrive/abuse/outputs/checkpoint-2000/vocab.json'
'/content/outputs/checkpoint-2000/merges.txt' -> '/content/drive/MyDrive/abuse/outputs/checkpoint-2000/merges.txt'
'/content/outputs/checkpoint-2000/tokenizer.json' -> '/content/drive/MyDrive/abuse/outputs/c